# Ballot measure analysis - Download cash

Download the cash for all of the petition measure sponsors, including the money they paid to a list of known signature-gathering companies.

In [1]:
import csv
import pandas as pd
import numpy as np

from sunscrape.committee import CommitteeScraper

## Load in the list of ballot measures

In [2]:
bm_df = pd.read_csv('data/ballot_measures.csv', encoding="ISO-8859-1")
bm_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496 entries, 0 to 495
Data columns (total 6 columns):
Elec Year             496 non-null object
Status                496 non-null object
Date                  494 non-null object
Title                 495 non-null object
Serial #(Ballot #)    408 non-null object
Sponsor               496 non-null object
dtypes: object(6)
memory usage: 23.4+ KB


In [3]:
bm_df.head()

,Elec Year,Status,Date,Title,Serial #(Ballot #),Sponsor
0,2020 GEN,Withdrawn,9/6/19,ADULT USE OF MARIJUANA,19-10,Make It Legal Florida
1,2020 GEN,Active,9/6/19,ADULT USE OF MARIJUANA,19-10,Make It Legal Florida
2,2020 GEN,Active,1/29/19,All drivers may complete community service in ...,19-02,"Drivers Ready to Improve, Vote and Empower (D...."
3,2020 GEN,Withdrawn,5/24/19,All registered voters must be allowed to vote ...,18-07,Florida Fair and Open Primaries
4,2020 GEN,Active,4/16/15,All tolls shall be approved by voters,15-07,"Drivers Ready to Improve, Vote and Empower (D...."


## Check sponsors

We have a list of known signature-gathering companies, so let's check each of the sponsors' expenses to see fi they've hired these companies.

In [4]:
# List of known companies
companies_list = [
    "Allied Data Service",
    "Arno Political Consultants",
    "Bader & Associates, Inc.",
    "BALLOT ACCESS, LLC",
    "BALLOT ACCESS",
    "Campaign Finance",
    "Democracy Resources",
    "Fieldworks",
    "Grassroots Campaigns Inc.",
    "LINJEN CORP",
    "LINJEN CORP.",
    "Jefferson Adams Consulting INC",
    "Kimball Petition Management",
    "LAMM Political Partners, LLC",
    "NATIONAL VOTER OUTREACH INC.",
    "Masterson & Wright",
    "National Ballot Access",
    "National Petition Management",
    "NATIONAL VOTER OUTREACH, INC.",
    "National Voter Outreach",
    "Petition Partners",
    "Progressive Campaigns, Inc.",
    "CAMPAIGN AUDIT AND TRUST",
    "Schumacher & Associates LLC",
    "Silver Bullet LLC",
    "Signature Masters Inc",
    "PCI Consultants, inc",
    "PCI Consultants",
    "PCI CONSULTANTS INC.",
    "PCI",
    "SILVER BULLET",
    "LET THE VOTERS DECIDE",
    "LET THE VOTER'S DECIDE",
    "LET THE VOTERS' DECIDE",
    "Arno Petition Consultants",
    "Progressive Campaigns",
    "AAP HOLDING COMPANY INC",
    "AAP HOLDING",
    "BH-AP PETITIONING",
    "AAP HOLDING CO, INC.",
    "THE FAIRNESS PROJECT",
    "AAP HOLDING COMPANY",
    "GRASSROOTS PETITIONERS",
    "AP PETITIONING PARTNERS",
    "SAHM PETITION",
    "AP PETITIONING PARTNERS LLC",
    "AWA STRATEGIES, INC.",
    "STRATEGIC INITIATIVE, LLC",
    "MIRANDA ADVOCACY, LLC",
    "SAHM PETITION MANAGEMENT",
    "KIMBALL PETITION MANAGEMENT"
]

### First, how much has each sponsor paid to these companies?

In [5]:
def check_sponsor_expenses(sponsor):
    
    if "The Florida Legislature" not in sponsor["Sponsor"] and sponsor["Sponsor"] != "Constitution Revision Commission":
        try:
            
            companies_str = "|".join([x.upper() for x in companies_list])


            comm_scraper = CommitteeScraper(committee_name=sponsor["Sponsor"][:50], result_type="expenditures")
            expense_df = pd.DataFrame(data=comm_scraper.results)

            committee_name = sponsor["Sponsor"].lower().replace(" ", "-")
            expense_df.to_csv("data/committee_expenses/{0}_expenses.csv".format(committee_name))

            grouped_expenses = expense_df.groupby("paid_to_name").amount.sum().reset_index()
            top_expenses = grouped_expenses.sort_values("amount", ascending=False).paid_to_name.tolist()

            filter1 = expense_df['paid_to_name'].str.contains(companies_str)
            matches_df = expense_df[filter1]
            hired = matches_df["paid_to_name"].tolist()
            total_spent = matches_df.amount.sum()
            print("Processing " + sponsor["Sponsor"], total_spent, expense_df.amount.sum())
            return expense_df.amount.sum(), set(hired), total_spent, set(top_expenses[:10])

        except:
            return "", "", "", ""
            print("Error with entry", sponsor["Sponsor"])
    
    else:
        return "", "", "", ""

In [6]:
bm_df[["sponsor_total_spent", "sig_companies_paid", "total_paid_to_sig_companies", "top_expenses"]] = bm_df.apply(check_sponsor_expenses, axis=1, result_type="expand")
bm_df.head()

Processing Make It Legal Florida 0.0 2657238.45
Processing Make It Legal Florida 0.0 2657238.45
Processing Drivers Ready to Improve, Vote and Empower (D.R.I.V.E) 0.0 786.94
Processing Florida Fair and Open Primaries 0.0 2228.8400000000006
Processing Drivers Ready to Improve, Vote and Empower (D.R.I.V.E) 0.0 786.94
Processing Drivers Ready to Improve, Vote and Empower (D.R.I.V.E) 0.0 786.94
Processing All Voters Vote, Inc. 6417791.62 6888895.92
Processing All Voters Vote, Inc. 6417791.62 6888895.92
Processing Florida Citizen Voters 2024645.6500000001 2445006.0100000002
Processing Pledge of Acknowledgement 0.0 150.0
Processing Pledge of Acknowledgement 0.0 150.0
Processing Peaceful Minds for Medical Marijuana 0.0 3829.92
Processing Floridians for Universal Background Checks 0.0 57850.0
Processing Floridians for Solar Choice, Inc. 1260931.55 1685274.73
Processing Peaceful Minds for Medical Marijuana 0.0 3829.92
Processing League of Voters, Extraordinaire 0.0 977.79
Processing Ban Assault 

Processing Derail the Bullet Train (DEBT) 1194426.66 2143761.46
Processing Floridians for Patient Protection 2196689.86 28647620.36
Processing Floridians for Patient Protection 2196689.86 28647620.36
Processing Floridians for Patient Protection 2196689.86 28647620.36
Processing Floridians for Patient Protection 2196689.86 28647620.36
Processing Florida Campaign for New Drug Policies 278250.0 756302.7999999999
Processing Citizens For A Fair Share, Inc. 0.0 8586500.53
Processing Victims Compensation Coalition 0.0 80792.28999999998
Processing Citizens for Public Integrity 144875.0 228715.41999999998
Processing Floridians for Humane Farms 0.0 1592686.5499999998
Processing Floridians For a Level Playing Field 483749.9 18045958.71
Processing Floridians for Political Choice 0.0 194.12
Processing People Over Politics 0.0 10750.0
Processing Floridians Representing Equity and Equality (FREE) 0.0 100505.03
Processing Floridians Representing Equity and Equality (FREE) 0.0 100505.03
Processing Flor

,Elec Year,Status,Date,Title,Serial #(Ballot #),Sponsor,sponsor_total_spent,sig_companies_paid,total_paid_to_sig_companies,top_expenses
0,2020 GEN,Withdrawn,9/6/19,ADULT USE OF MARIJUANA,19-10,Make It Legal Florida,2.65724e+06,{},0,"{ROBERT WATKINS & COMPANY, P.A., COREMESSAGE, ..."
1,2020 GEN,Active,9/6/19,ADULT USE OF MARIJUANA,19-10,Make It Legal Florida,2.65724e+06,{},0,"{ROBERT WATKINS & COMPANY, P.A., COREMESSAGE, ..."
2,2020 GEN,Active,1/29/19,All drivers may complete community service in ...,19-02,"Drivers Ready to Improve, Vote and Empower (D....",786.94,{},0,"{MIAMI PRINTING, UNITED STATES POSTAL SERVICE,..."
3,2020 GEN,Withdrawn,5/24/19,All registered voters must be allowed to vote ...,18-07,Florida Fair and Open Primaries,2228.84,{},0,"{FACEBOOK, CULLEN TOM, HEWLETT-PACKARD, FACEBO..."
4,2020 GEN,Active,4/16/15,All tolls shall be approved by voters,15-07,"Drivers Ready to Improve, Vote and Empower (D....",786.94,{},0,"{MIAMI PRINTING, UNITED STATES POSTAL SERVICE,..."


### Second, how much has each sponsor raised from these companies?

Signature gethering help can also come in the form of in-kind contributions, so check those for the sponsors.

In [7]:
def check_sponsor_contribs(sponsor):
    
    if "The Florida Legislature" not in sponsor["Sponsor"] and sponsor["Sponsor"] != "Constitution Revision Commission":
        try:
            
            companies_str = "|".join([x.upper() for x in companies_list])


            comm_scraper = CommitteeScraper(committee_name=sponsor["Sponsor"][:50], result_type="contributions")
            contrib_df = pd.DataFrame(data=comm_scraper.results)

            committee_name = sponsor["Sponsor"].lower().replace(" ", "-")
            contrib_df.to_csv("data/committee_contribs/{0}_contribs.csv".format(committee_name))
            print(sponsor["Sponsor"])

            if contrib_df.empty:
                total_raised = 0
                inkind_donor = ""
                sigcos_total_raised = 0
                top_donors = ""

            else:
                grouped_contribs = contrib_df.groupby("contributor_name").amount.sum().reset_index()
                grouped_contribs_sorted = grouped_contribs.sort_values("amount", ascending=False).contributor_name.tolist()
                top_donors = set(grouped_contribs_sorted[:10])
                filter1 = contrib_df['contributor_name'].str.contains(companies_str)
                matches_df = contrib_df[filter1]
                inkind_donor = set(matches_df["contributor_name"].tolist())
                sigcos_total_raised = matches_df.amount.sum()
                total_raised = contrib_df.amount.sum()

            return total_raised, inkind_donor, sigcos_total_raised, top_donors
    #         print(contrib_df.amount.sum(), set(inkind_donor), sigcos_total_raised, set(top_donors[:10]))


        except:
            return "", "", "", ""
            print("Error with entry", sponsor["Sponsor"])
    
    else:
        return 0, "", 0, ""

In [8]:
bm_df[["sponsor_total_raised", "sig_company_donors", "total_raised_from_sig_companies", "top_donors"]] = bm_df.apply(check_sponsor_contribs, axis=1, result_type="expand")
bm_df.head()

Make It Legal Florida
Make It Legal Florida
Drivers Ready to Improve, Vote and Empower (D.R.I.V.E)
Florida Fair and Open Primaries
Drivers Ready to Improve, Vote and Empower (D.R.I.V.E)
Drivers Ready to Improve, Vote and Empower (D.R.I.V.E)
All Voters Vote, Inc.
All Voters Vote, Inc.
Florida Citizen Voters
Pledge of Acknowledgement
Pledge of Acknowledgement
Peaceful Minds for Medical Marijuana
Floridians for Universal Background Checks
Florida Faith Amendment .com
Floridians for Solar Choice, Inc.
Peaceful Minds for Medical Marijuana
League of Voters, Extraordinaire
Floridians for Redeemable People
Floridians for Redeemable People
Ban Assault Weapons Now
Ban Assault Weapons Now
Florida Coalition for Private Property Rights
Florida Decides Healthcare, Inc.
Florida For A Fair Wage
Committee To Amend To Recall
Sensible Florida, Inc.
Keep Our Constitution Clean PC
Floridians for Freedom
Citizens for Energy Choices
Floridians For Health Security
Personhood Florida
Florida Fair and Open Prim

Fish and Wildlife Conservation Committee
Florida Locally Approved Gaming, Inc.
Fla-187 Committee, Inc.
Fla-187 Committee, Inc.
Tax Cap Committee
Citizens for Campaign and Government Spending Reform
Tax Cap Committee
Political Committee for Florida Workers' Advocates
Teachers for Better Public Schools
Floridians for Health Care Choice
Political Committee for Florida Workers' Advocates
Stop Early Release Committee (STOP)
The Florida Civil Rights Initiative
Committee to Lower and Equalize Contingent Attorney Fees
Tax Cap Committee
The Campaign For Florida's Future
The Campaign For Florida's Future
Florida State Lodge, Fraternal Order of Police
The Safe Bet for Florida Committee
The Campaign For Florida's Future
Independence Committee
The Parole Elimination Network Group
Save Our Everglades Committee
Save Our Everglades Committee
Enough is Enough Committee
The American Dream in Florida PC
Save Our Everglades Committee
Florida Pension Reform Committee
Tax Cap Committee
Take Back America
Pro

,Elec Year,Status,Date,Title,Serial #(Ballot #),Sponsor,sponsor_total_spent,sig_companies_paid,total_paid_to_sig_companies,top_expenses,sponsor_total_raised,sig_company_donors,total_raised_from_sig_companies,top_donors
0,2020 GEN,Withdrawn,9/6/19,ADULT USE OF MARIJUANA,19-10,Make It Legal Florida,2.65724e+06,{},0,"{ROBERT WATKINS & COMPANY, P.A., COREMESSAGE, ...",2.81956e+06,{},0,"{DALY THOMAS, POLLACK GREGG, HAWES KEN, RAMOS ..."
1,2020 GEN,Active,9/6/19,ADULT USE OF MARIJUANA,19-10,Make It Legal Florida,2.65724e+06,{},0,"{ROBERT WATKINS & COMPANY, P.A., COREMESSAGE, ...",2.81956e+06,{},0,"{DALY THOMAS, POLLACK GREGG, HAWES KEN, RAMOS ..."
2,2020 GEN,Active,1/29/19,All drivers may complete community service in ...,19-02,"Drivers Ready to Improve, Vote and Empower (D....",786.94,{},0,"{MIAMI PRINTING, UNITED STATES POSTAL SERVICE,...",1131.95,{},0,"{MIAMI PRINTING, LEMUS TONY, WILLS ISABEL, BUC..."
3,2020 GEN,Withdrawn,5/24/19,All registered voters must be allowed to vote ...,18-07,Florida Fair and Open Primaries,2228.84,{},0,"{FACEBOOK, CULLEN TOM, HEWLETT-PACKARD, FACEBO...",4499.85,{},0,"{HOUGH STEVEN, SOLOMON JEFFREY, BROUGHTON TERR..."
4,2020 GEN,Active,4/16/15,All tolls shall be approved by voters,15-07,"Drivers Ready to Improve, Vote and Empower (D....",786.94,{},0,"{MIAMI PRINTING, UNITED STATES POSTAL SERVICE,...",1131.95,{},0,"{MIAMI PRINTING, LEMUS TONY, WILLS ISABEL, BUC..."


## Save compiled 

In [9]:
bm_df.to_csv("ballot_petition_stats.csv")